In [1]:
import pandas as pd
import numpy as np

import time, warnings
import datetime as dt

import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn import preprocessing
%matplotlib inline

In [2]:
raw_data = pd.read_csv("C:/Users/User/Desktop/nida/semester/summer/for git hub/supermarket.csv",dtype={'SHOP_DATE': str})
raw_data.head()

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,...,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
0,200734,20071019,6,8,1,9.50,PRD0901335,CL00058,DEP00016,G00006,...,XX,OT,994108000679877,M,MM,Small Shop,Fresh,STORE00001,LS,E02
1,200805,20080328,6,21,1,1.49,PRD0901120,CL00162,DEP00055,G00016,...,XX,NaN,994110300629787,S,MM,Small Shop,Fresh,STORE00001,LS,E02
2,200716,20070612,3,9,1,0.86,PRD0900963,CL00248,DEP00089,G00030,...,NaN,NaN,994106200026009,S,MM,Small Shop,Grocery,STORE00001,LS,E02
3,200815,20080604,4,10,3,0.90,PRD0901055,CL00248,DEP00089,G00030,...,NaN,NaN,994111300076346,L,MM,Top Up,Grocery,STORE00001,LS,E02
4,200626,20060824,5,16,1,1.77,PRD0901877,CL00169,DEP00056,G00017,...,NaN,NaN,994102000065854,L,LA,Top Up,Fresh,STORE00001,LS,E02


In [3]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578082 entries, 0 to 578081
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SHOP_WEEK                 578082 non-null  int64  
 1   SHOP_DATE                 578082 non-null  object 
 2   SHOP_WEEKDAY              578082 non-null  int64  
 3   SHOP_HOUR                 578082 non-null  int64  
 4   QUANTITY                  578082 non-null  int64  
 5   SPEND                     578082 non-null  float64
 6   PROD_CODE                 578082 non-null  object 
 7   PROD_CODE_10              578082 non-null  object 
 8   PROD_CODE_20              578082 non-null  object 
 9   PROD_CODE_30              578082 non-null  object 
 10  PROD_CODE_40              578082 non-null  object 
 11  CUST_CODE                 492494 non-null  object 
 12  CUST_PRICE_SENSITIVITY    492494 non-null  object 
 13  CUST_LIFESTAGE            433142 non-null  o

In [4]:
data = raw_data.sort_values(by=['QUANTITY'],ascending=True)

In [5]:
data.isnull().sum()

SHOP_WEEK                        0
SHOP_DATE                        0
SHOP_WEEKDAY                     0
SHOP_HOUR                        0
QUANTITY                         0
SPEND                            0
PROD_CODE                        0
PROD_CODE_10                     0
PROD_CODE_20                     0
PROD_CODE_30                     0
PROD_CODE_40                     0
CUST_CODE                    85588
CUST_PRICE_SENSITIVITY       85588
CUST_LIFESTAGE              144940
BASKET_ID                        0
BASKET_SIZE                      0
BASKET_PRICE_SENSITIVITY         0
BASKET_TYPE                      0
BASKET_DOMINANT_MISSION          0
STORE_CODE                       0
STORE_FORMAT                     0
STORE_REGION                     0
dtype: int64

In [6]:
#remove rows where Customer ID are Na
data.dropna(subset=['CUST_CODE'],how='all',inplace=True)
data.shape

(492494, 22)

In [7]:
data.head()

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,...,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
0,200734,20071019,6,8,1,9.50,PRD0901335,CL00058,DEP00016,G00006,...,XX,OT,994108000679877,M,MM,Small Shop,Fresh,STORE00001,LS,E02
365497,200740,20071126,2,16,1,2.23,PRD0901619,CL00058,DEP00016,G00006,...,LA,YF,994108600736506,L,LA,Full Shop,Mixed,STORE00001,LS,E02
365496,200819,20080701,3,21,1,4.54,PRD0904362,CL00058,DEP00016,G00006,...,LA,YF,994111700816636,L,MM,Full Shop,Fresh,STORE00001,LS,E02
365495,200752,20080221,5,20,1,7.13,PRD0900324,CL00058,DEP00016,G00006,...,LA,YF,994109800783283,L,LA,Top Up,Mixed,STORE00001,LS,E02
365494,200624,20060810,5,18,1,3.21,PRD0900109,CL00059,DEP00016,G00006,...,LA,OA,994101800339486,L,LA,Top Up,Fresh,STORE00001,LS,E02


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 492494 entries, 0 to 14690
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SHOP_WEEK                 492494 non-null  int64  
 1   SHOP_DATE                 492494 non-null  object 
 2   SHOP_WEEKDAY              492494 non-null  int64  
 3   SHOP_HOUR                 492494 non-null  int64  
 4   QUANTITY                  492494 non-null  int64  
 5   SPEND                     492494 non-null  float64
 6   PROD_CODE                 492494 non-null  object 
 7   PROD_CODE_10              492494 non-null  object 
 8   PROD_CODE_20              492494 non-null  object 
 9   PROD_CODE_30              492494 non-null  object 
 10  PROD_CODE_40              492494 non-null  object 
 11  CUST_CODE                 492494 non-null  object 
 12  CUST_PRICE_SENSITIVITY    492494 non-null  object 
 13  CUST_LIFESTAGE            433142 non-null  ob

In [9]:
data.SHOP_DATE.min()

'20060410'

In [10]:
data.SHOP_DATE.max()

'20080706'

In [11]:
#ewatrict the data to one full year beacuse it's better to use a metric per months or years in RFM
data['SHOP_DATE'] = pd.to_datetime(data['SHOP_DATE'])
startdate = pd.to_datetime('20070706')
data = data[data['SHOP_DATE'] >= startdate]
data.head()

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,...,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
0,200734,2007-10-19,6,8,1,9.50,PRD0901335,CL00058,DEP00016,G00006,...,XX,OT,994108000679877,M,MM,Small Shop,Fresh,STORE00001,LS,E02
365497,200740,2007-11-26,2,16,1,2.23,PRD0901619,CL00058,DEP00016,G00006,...,LA,YF,994108600736506,L,LA,Full Shop,Mixed,STORE00001,LS,E02
365496,200819,2008-07-01,3,21,1,4.54,PRD0904362,CL00058,DEP00016,G00006,...,LA,YF,994111700816636,L,MM,Full Shop,Fresh,STORE00001,LS,E02
365495,200752,2008-02-21,5,20,1,7.13,PRD0900324,CL00058,DEP00016,G00006,...,LA,YF,994109800783283,L,LA,Top Up,Mixed,STORE00001,LS,E02
365491,200733,2007-10-09,3,15,1,7.16,PRD0902503,CL00058,DEP00016,G00006,...,LA,OF,994107900231660,L,LA,Full Shop,Mixed,STORE00001,LS,E02


In [12]:
print('summary')
#exporting the unique values of each attribute
print('Number of transaction: ', data.groupby('SHOP_DATE')['SHOP_DATE'].nunique().count())
print('Number of products ', data.groupby('PROD_CODE')['PROD_CODE'].nunique().count())
print('Number of customers: ', data.groupby('CUST_CODE')['CUST_CODE'].nunique().count())
print('Percentage if customers NA: ', round(data['CUST_CODE'].isnull().sum() * 100 / len(data), 2), "%")

summary
Number of transaction:  367
Number of products  4880
Number of customers:  2533
Percentage if customers NA:  0.0 %


In [13]:
now = dt.date(2008,7,6)
print(now)

2008-07-06


In [14]:
#groupby customer and check last date of purshace
recency_df = data.groupby(by='CUST_CODE',as_index=False)['SHOP_DATE'].max()
recency_df.columns = ['CUST_CODE','LastPurshaceDate']
# Convert the 'LastPurshaceDate' column to datetime.date objects
recency_df['LastPurshaceDate'] = pd.to_datetime(recency_df['LastPurshaceDate']).dt.date
recency_df.head()

,CUST_CODE,LastPurshaceDate
0,CUST0000000689,2008-06-23
1,CUST0000001163,2008-06-22
2,CUST0000001194,2008-06-20
3,CUST0000002605,2008-06-12
4,CUST0000002637,2008-05-30


In [15]:
import pandas as pd
from datetime import datetime

# Assuming you have a DataFrame 'recency_df' with a 'LastPurshaceDate' column



# Calculate recency in days
recency_df['Recency'] = recency_df['LastPurshaceDate'].apply(lambda x : (now - x).days) #(now - recency_df['LastPurshaceDate']).dt.days

# Print the DataFrame
print(recency_df)


           CUST_CODE LastPurshaceDate  Recency
0     CUST0000000689       2008-06-23       13
1     CUST0000001163       2008-06-22       14
2     CUST0000001194       2008-06-20       16
3     CUST0000002605       2008-06-12       24
4     CUST0000002637       2008-05-30       37
...              ...              ...      ...
2528  CUST0000999024       2008-06-03       33
2529  CUST0000999439       2008-07-05        1
2530  CUST0000999593       2008-04-02       95
2531  CUST0000999935       2008-06-05       31
2532  CUST0000999936       2008-04-17       80

[2533 rows x 3 columns]


In [16]:
#drop duplicates
data_copy = data
data_copy.drop_duplicates(subset= ['BASKET_ID','CUST_CODE'],keep ='first',inplace=True)

#calculate Frequency of purchases
frequency_df = data_copy.groupby(by=['CUST_CODE'], as_index=False)['BASKET_ID'].count()
frequency_df.columns = ['CUST_CODE','Freqyency']
frequency_df.head()

,CUST_CODE,Freqyency
0,CUST0000000689,16
1,CUST0000001163,1
2,CUST0000001194,55
3,CUST0000002605,2
4,CUST0000002637,4


In [17]:
monetary_df = data.groupby(['CUST_CODE'], as_index=False).agg({'SPEND': 'sum'})
monetary_df.columns = ['CUST_CODE','Monetary']
print(monetary_df.head())

        CUST_CODE  Monetary
0  CUST0000000689     20.39
1  CUST0000001163      0.52
2  CUST0000001194    103.39
3  CUST0000002605      0.60
4  CUST0000002637      7.76


In [18]:
#merge recency dataframe with the frequency dataframe
temp_df = recency_df.merge(frequency_df, on='CUST_CODE')
temp_df.head()

,CUST_CODE,LastPurshaceDate,Recency,Freqyency
0,CUST0000000689,2008-06-23,13,16
1,CUST0000001163,2008-06-22,14,1
2,CUST0000001194,2008-06-20,16,55
3,CUST0000002605,2008-06-12,24,2
4,CUST0000002637,2008-05-30,37,4


In [19]:
#merge with monrtary dataframe to get a table with the 3 columns
rfm_df = temp_df.merge(monetary_df, on ='CUST_CODE')
#use CUST_CODE as index
rfm_df.set_index('CUST_CODE',inplace = True)
#check the head
rfm_df.head()

,LastPurshaceDate,Recency,Freqyency,Monetary
CUST_CODE,,,,
CUST0000000689,2008-06-23,13,16,20.39
CUST0000001163,2008-06-22,14,1,0.52
CUST0000001194,2008-06-20,16,55,103.39
CUST0000002605,2008-06-12,24,2,0.60
CUST0000002637,2008-05-30,37,4,7.76


In [20]:
tertiles = rfm_df.quantile(q=[0.33,0.66])
tertiles = rfm_df.quantile(q=[0.33,0.66])

In [21]:
tertiles.to_dict()

{'Recency': {0.33: 23.0, 0.66: 109.0},
 'Freqyency': {0.33: 1.0, 0.66: 6.0},
 'Monetary': {0.33: 2.0300000000000002, 0.66: 10.002400000000002}}

In [22]:
#Arguments (x =value , p recency , monetary_value , frequency, d =tertiles dict)
def RScore(x,p,d):
    if x <= d[p][0.33]:
        return 3
    elif x <= d[p][0.66]:
        return 2
    else:
        return 1
#Arguments (x =value , p recency , monetary_value , frequency, k =tertiles dict)
def FMScore(x,p,k):
    if x <= k[p][0.33]:
        return 1
    elif x <= k[p][0.66]:
        return 2
    else:
        return 3

In [23]:
#Create rfm segentation table/tambah column based on formula
rfm_segmentation = rfm_df
rfm_segmentation['R_tertile'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',tertiles,))
rfm_segmentation['F_tertile'] = rfm_segmentation['Freqyency'].apply(RScore, args=('Freqyency',tertiles,))
rfm_segmentation['M_tertile'] = rfm_segmentation['Monetary'].apply(RScore, args=('Monetary',tertiles,))
rfm_segmentation.head()

,LastPurshaceDate,Recency,Freqyency,Monetary,R_tertile,F_tertile,M_tertile
CUST_CODE,,,,,,,
CUST0000000689,2008-06-23,13,16,20.39,3,1,1
CUST0000001163,2008-06-22,14,1,0.52,3,3,3
CUST0000001194,2008-06-20,16,55,103.39,3,1,1
CUST0000002605,2008-06-12,24,2,0.60,2,2,3
CUST0000002637,2008-05-30,37,4,7.76,2,2,2


In [24]:
rfm_segmentation['RFM_Score'] = rfm_segmentation.R_tertile.map(str)\
+rfm_segmentation.F_tertile.map(str)\
+rfm_segmentation.M_tertile.map(str)
rfm_segmentation

,LastPurshaceDate,Recency,Freqyency,Monetary,R_tertile,F_tertile,M_tertile,RFM_Score
CUST_CODE,,,,,,,,
CUST0000000689,2008-06-23,13,16,20.39,3,1,1,311
CUST0000001163,2008-06-22,14,1,0.52,3,3,3,333
CUST0000001194,2008-06-20,16,55,103.39,3,1,1,311
CUST0000002605,2008-06-12,24,2,0.60,2,2,3,223
CUST0000002637,2008-05-30,37,4,7.76,2,2,2,222
...,...,...,...,...,...,...,...,...
CUST0000999024,2008-06-03,33,13,34.35,2,1,1,211
CUST0000999439,2008-07-05,1,5,18.01,3,2,1,321
CUST0000999593,2008-04-02,95,11,16.45,2,1,1,211


In [25]:
import requests
from io import StringIO


In [26]:
maxs = rfm_segmentation['RFM_Score'].max()
mins = rfm_segmentation['RFM_Score'].min()
print(maxs, mins)

333 111


In [50]:
rfm_segmentation['RFM_Score'].unique()


array(['311', '333', '223', '222', '211', '122', '322', '233', '133',
       '123', '212', '323', '232', '321', '132', '121', '221', '131',
       '312', '231', '111', '332', '112'], dtype=object)

In [38]:
rfm_segmentation.drop_duplicates().sort_values("RFM_Score")

,LastPurshaceDate,Recency,Freqyency,Monetary,R_tertile,F_tertile,M_tertile,RFM_Score,segment_name
CUST_CODE,,,,,,,,,
CUST0000303829,2008-03-14,114,12,15.90,1,1,1,111,Lost Customer
CUST0000383763,2008-03-02,126,19,28.11,1,1,1,111,Lost Customer
CUST0000413889,2007-12-27,192,8,19.13,1,1,1,111,Lost Customer
CUST0000703478,2008-03-15,113,7,21.43,1,1,1,111,Lost Customer
CUST0000299156,2007-12-18,201,7,12.63,1,1,1,111,Lost Customer
...,...,...,...,...,...,...,...,...,...
CUST0000374978,2008-06-20,16,1,1.65,3,3,3,333,Campion
CUST0000192879,2008-06-26,10,1,1.02,3,3,3,333,Campion
CUST0000275219,2008-06-27,9,1,1.69,3,3,3,333,Campion


In [41]:
segment_name = {'112':'About to sleep',
                '113':'About to sleep',
                '121':'About to sleep',
                '122':'About to sleep',
                '131':'About to sleep',
                '211':'About to sleep',
                '212':'About to sleep',
                '221':'About to sleep',
                
                '311':'Recent customer',
                '312':'Recent customer',
                '313':'Recent customer',
                
                '123':'Cannot Loose them',
                '132':'Cannot Loose them',
                '133':'Cannot Loose them',
                
                '333':'Campion',
                '111':'Lost Customer',
                
                '233':'Loyal',
                '323':'Loyal',
                '332':'Loyal',
                '232':'Loyal',
                
                '223':'potential Lotalist',
                '322':'potential Lotalist',
                '331':'potential Lotalist',
                '213':'Average',
               '222':'Average',
               '231':'Average',
               '321':'Average'}

In [42]:
rfm_segmentation['segment_name'] = rfm_segmentation["RFM_Score"].map(segment_name)

In [51]:
#for check segment_name is NaN or not
rfm_segmentation['segment_name'].unique()

array(['Recent customer', 'Campion', 'potential Lotalist', 'Average',
       'About to sleep', 'Loyal', 'Cannot Loose them', 'Lost Customer'],
      dtype=object)

In [47]:
rfm_segmentation

,LastPurshaceDate,Recency,Freqyency,Monetary,R_tertile,F_tertile,M_tertile,RFM_Score,segment_name
CUST_CODE,,,,,,,,,
CUST0000000689,2008-06-23,13,16,20.39,3,1,1,311,Recent customer
CUST0000001163,2008-06-22,14,1,0.52,3,3,3,333,Campion
CUST0000001194,2008-06-20,16,55,103.39,3,1,1,311,Recent customer
CUST0000002605,2008-06-12,24,2,0.60,2,2,3,223,potential Lotalist
CUST0000002637,2008-05-30,37,4,7.76,2,2,2,222,Average
...,...,...,...,...,...,...,...,...,...
CUST0000999024,2008-06-03,33,13,34.35,2,1,1,211,About to sleep
CUST0000999439,2008-07-05,1,5,18.01,3,2,1,321,Average
CUST0000999593,2008-04-02,95,11,16.45,2,1,1,211,About to sleep


In [52]:
#Export the data to csv, and you can visualized on Tableau or PowerBI
#rfm_segmentation.to_csv('rfm_segment.csv')

In [ ]:
CUST_PRICE_SENSITIVITY
CUST_LIFESTAGE
SPEND
RFM_Score